In [2]:
#Import libraries
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json # library to handle JSON files
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
#!conda install -c conda-forge geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium
import folium # map rendering library

print('Libraries imported!')

Libraries imported!


## Get the postal code data

In [3]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Seattle').text

In [4]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [5]:
# create a list to store neighborhood data
neighborhoodList = []

In [6]:
# Extract neighborhood names from the data
for row in soup.findAll("table")[0].findAll("tr"):
    cells=row.findAll('td')
    if(len(cells)>0):
        neighborhoodList.append(cells[0].text.rstrip('\n'))


## Create a dataframe that includes neighborhoods

In [7]:
# create a new DataFrame from the list
seattle_df = pd.DataFrame({"Neighborhood": neighborhoodList})
seattle_df.head()

,Neighborhood
0,North Seattle
1,Broadview
2,Bitter Lake
3,North Beach / Blue Ridge
4,Crown Hill


In [8]:
# print the number of rows of the dataframe
seattle_df.shape

(127, 1)

In [9]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Seattle, Washington'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in seattle_df["Neighborhood"].tolist() ]
coords

[[47.56232264198057, -122.31069995446725],
 [47.722380000000044, -122.36497999999995],
 [47.71868000000006, -122.35029999999995],
 [47.70047000000005, -122.38419999999996],
 [47.69494000000003, -122.37409999999994],
 [47.69082000000003, -122.35528999999997],
 [47.713100000000054, -122.31929999999994],
 [47.72320000000008, -122.33869999999996],
 [47.71894000000003, -122.31399999999996],
 [47.69912000000005, -122.33967999999999],
 [47.699940000000026, -122.31764999999996],
 [47.71780000000007, -122.28317999999996],
 [47.72645000000006, -122.28787999999997],
 [47.69934000000006, -122.27834999999999],
 [47.70841000000007, -122.29582999999997],
 [47.72655000000003, -122.30258999999995],
 [47.710300000000075, -122.30719999999997],
 [47.68701000000004, -122.29493999999994],
 [47.68031000000008, -122.27297999999996],
 [47.682120000000054, -122.26080999999999],
 [47.68156000000005, -122.31694999999996],
 [47.67396000000008, -122.29903999999999],
 [47.67579000000006, -122.28718999999995],
 [47.6

In [11]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
seattle_df['Latitude'] = df_coords['Latitude']
seattle_df['Longitude'] = df_coords['Longitude']

# check the neighborhoods and the coordinates
print(seattle_df.shape)
seattle_df

(127, 3)


,Neighborhood,Latitude,Longitude
0,North Seattle,47.562323,-122.310700
1,Broadview,47.722380,-122.364980
2,Bitter Lake,47.718680,-122.350300
3,North Beach / Blue Ridge,47.700470,-122.384200
4,Crown Hill,47.694940,-122.374100
5,Greenwood,47.690820,-122.355290
6,Northgate,47.713100,-122.319300
7,Haller Lake,47.723200,-122.338700
8,Pinehurst,47.718940,-122.314000
9,North College Park\n(Licton Springs),47.699120,-122.339680


In [12]:
# save the DataFrame as CSV file
seattle_df.to_csv("seattle_df.csv", index=False)

In [15]:
# get the coordinates
address = 'Seattle, Washington'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Seattle, Washington {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Seattle, Washington 47.6038321, -122.3300624.


In [16]:
# create map of Seattle using latitude and longitude values
map_seattle = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(seattle_df['Latitude'], seattle_df['Longitude'], seattle_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_seattle)  
    
map_seattle

In [17]:
# save the map as HTML file
map_seattle.save('map_seattle.html')

In [18]:
# define Foursquare Credentials and Version
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [19]:
radius = 2000
LIMIT = 100
venues = []
for lat, long, neighborhood in zip(seattle_df['Latitude'], seattle_df['Longitude'], seattle_df['Neighborhood']):
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [20]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)
# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(11493, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,North Seattle,47.562323,-122.3107,MacPherson's Fruit & Produce,47.563328,-122.313276,Farmers Market
1,North Seattle,47.562323,-122.3107,Jefferson Park,47.567405,-122.310362,Park
2,North Seattle,47.562323,-122.3107,Breezy Town Pizza,47.559093,-122.304091,Pizza Place
3,North Seattle,47.562323,-122.3107,Jefferson Park Driving Range,47.567232,-122.307457,Golf Driving Range
4,North Seattle,47.562323,-122.3107,Jack's BBQ,47.568289,-122.322433,BBQ Joint


In [21]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adams,100,100,100,100,100,100
Alki Point,71,71,71,71,71,71
Arbor Heights,97,97,97,97,97,97
Atlantic,100,100,100,100,100,100
Ballard,100,100,100,100,100,100
Beacon Hill,100,100,100,100,100,100
Belltown,100,100,100,100,100,100
Bitter Lake,91,91,91,91,91,91
Briarcliff,56,56,56,56,56,56


In [22]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 348 uniques categories.


In [23]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Farmers Market', 'Park', 'Pizza Place', 'Golf Driving Range',
       'BBQ Joint', 'Playground', 'Pub', 'Golf Course', 'Bar',
       'Mediterranean Restaurant', 'Mexican Restaurant', 'Lounge',
       'Brewery', 'Bookstore', 'Dim Sum Restaurant', 'Coffee Shop',
       'Beer Garden', 'Chocolate Shop', 'Food Truck', 'Café',
       'Taco Place', 'Ethiopian Restaurant', 'Arts & Crafts Store',
       'Diner', 'Bakery', 'Caribbean Restaurant', 'Restaurant',
       'Fried Chicken Joint', 'Design Studio', 'Indie Movie Theater',
       'Flea Market', 'Record Shop', 'Pet Store', 'Italian Restaurant',
       'Food Court', 'Greek Restaurant', 'Asian Restaurant',
       'Vietnamese Restaurant', 'Thai Restaurant', 'Dive Bar',
       'French Restaurant', 'Gastropub', 'Gymnastics Gym',
       'Fish & Chips Shop', 'Szechuan Restaurant', 'Spa',
       'African Restaurant', 'Ice Cream Shop', 'Garden Center',
       'Smoke Shop'], dtype=object)

In [24]:
# one hot encoding
seattle_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
seattle_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [seattle_onehot.columns[-1]] + list(seattle_onehot.columns[:-1])
seattle_onehot = seattle_onehot[fixed_columns]

print(seattle_onehot.shape)
seattle_onehot.head()

(11493, 349)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Terminal,Alternative Healer,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Carpet Store,Casino,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Science Building,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Fabric Shop,Fair,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Heliport,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Inn,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Pub,Public Art,Radio Station,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science M

In [25]:
seattle_grouped = seattle_onehot.groupby(["Neighborhoods"]).mean().reset_index()
print(seattle_grouped.shape)
seattle_grouped

(127, 349)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Terminal,Alternative Healer,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Carpet Store,Casino,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Science Building,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Fabric Shop,Fair,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Heliport,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Inn,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Pub,Public Art,Radio Station,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science M

In [26]:
len(seattle_grouped[seattle_grouped["Coffee Shop"] > 0])

124

# Create a data frame for Seattle coffee shops

In [27]:
seattle_coffeeshops = seattle_grouped[["Neighborhoods","Coffee Shop"]]

In [28]:
seattle_coffeeshops.head()

,Neighborhoods,Coffee Shop
0,Adams,0.050000
1,Alki Point,0.098592
2,Arbor Heights,0.061856
3,Atlantic,0.080000
4,Ballard,0.040000


# Cluster neighborhoods

In [29]:
# set number of clusters
kclusters = 3

kl_clustering = seattle_coffeeshops.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 0, 1, 2, 1, 2, 0, 0, 2])

In [30]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
seattle_merged = seattle_coffeeshops.copy()

# add clustering labels
seattle_merged["Cluster Labels"] = kmeans.labels_

In [31]:
seattle_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
seattle_merged.head()

,Neighborhood,Coffee Shop,Cluster Labels
0,Adams,0.050000,0
1,Alki Point,0.098592,1
2,Arbor Heights,0.061856,0
3,Atlantic,0.080000,1
4,Ballard,0.040000,2


In [32]:
seattle_merged = seattle_merged.join(seattle_df.set_index("Neighborhood"), on="Neighborhood")

print(seattle_merged.shape)
seattle_merged.head() # check the last columns!

(127, 5)


,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Adams,0.050000,0,47.672970,-122.387600
1,Alki Point,0.098592,1,47.582391,-122.373317
2,Arbor Heights,0.061856,0,47.508610,-122.375930
3,Atlantic,0.080000,1,47.596010,-122.302230
4,Ballard,0.040000,2,47.668670,-122.384530


In [33]:
# sort the results by Cluster Labels
print(seattle_merged.shape)
seattle_merged.sort_values(["Cluster Labels"], inplace=True)
seattle_merged

(127, 5)


,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Adams,0.050000,0,47.672970,-122.387600
41,Harrison[120] / Denny-Blaine[120],0.059701,0,46.721673,-122.968670
42,Hawthorne Hills,0.047619,0,47.549305,-122.261670
44,Highland Park,0.070000,0,47.529870,-122.351690
47,Industrial District,0.070000,0,47.586160,-122.327380
51,Judkins Park,0.060000,0,47.594270,-122.304010
55,Lakewood,0.050000,0,47.571410,-122.281803
60,Loyal Heights,0.060000,0,47.683930,-122.384310
62,Madison Valley,0.070000,0,47.607504,-122.329582
125,Windermere,0.053571,0,47.669990,-122.266270


In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(seattle_merged['Latitude'], seattle_merged['Longitude'], seattle_merged['Neighborhood'], seattle_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

# Examine clusters

## Cluster 0

In [36]:
seattle_merged.loc[seattle_merged['Cluster Labels'] == 0]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Adams,0.050000,0,47.672970,-122.387600
41,Harrison[120] / Denny-Blaine[120],0.059701,0,46.721673,-122.968670
42,Hawthorne Hills,0.047619,0,47.549305,-122.261670
44,Highland Park,0.070000,0,47.529870,-122.351690
47,Industrial District,0.070000,0,47.586160,-122.327380
51,Judkins Park,0.060000,0,47.594270,-122.304010
55,Lakewood,0.050000,0,47.571410,-122.281803
60,Loyal Heights,0.060000,0,47.683930,-122.384310
62,Madison Valley,0.070000,0,47.607504,-122.329582
125,Windermere,0.053571,0,47.669990,-122.266270


## Cluster 1

In [37]:
seattle_merged.loc[seattle_merged['Cluster Labels'] == 1]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
17,"Central Area[120] / Central District (""CD"")",0.080000,1,47.605530,-122.334320
85,Pigeon Point,0.120482,1,47.561510,-122.360260
79,North Queen Anne,0.090000,1,47.645860,-122.368400
90,Portage Bay[95] / Roanoke,0.090000,1,47.646900,-122.320220
78,North Delridge,0.110000,1,47.564730,-122.363860
77,North College Park\n(Licton Springs),0.080000,1,47.699120,-122.339680
15,"Cascade, Seattle",0.100000,1,47.603570,-122.329450
14,Capitol Hill,0.080000,1,47.623960,-122.318820
82,Northlake,0.080000,1,47.655038,-122.320812
84,Phinney Ridge,0.080000,1,47.675960,-122.354360


## Cluster 2

In [38]:
seattle_merged.loc[seattle_merged['Cluster Labels'] == 2]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
119,West Edge[118][119],0.000000,2,47.496791,-122.336009
108,South Park,0.021277,2,47.527040,-122.313760
48,Interbay,0.040000,2,47.653990,-122.379660
46,Holly Park[144] / NewHolly[145],0.026667,2,47.539760,-122.287990
4,Ballard,0.040000,2,47.668670,-122.384530
114,Victory Heights,0.030000,2,47.710300,-122.307200
106,South End,0.030000,2,47.562158,-122.307787
6,Belltown,0.040000,2,47.615760,-122.344640
74,North Admiral[153] / Admiral District,0.000000,2,42.744249,-83.055123
100,Sand Point,0.000000,2,47.682120,-122.260810


### Analyzing the three clusters above clearly shows that there are more coffee places concentration in the neighborhoods mentioned in Cluster 1, and the least in Cluster 2. Hence, the neighborhoods in Cluster 2 have good potential of opening a new profitable coffee place, given less competition. 